In [1]:
from Utilities.FileCollector import FileCollector
from Utilities.ShiftRecognizer import ShiftRecognizer
from Utilities.CycleRecognizer import CycleRecognizer
import pandas

# Define paths
input_path = 'WI3_BusinessIntelligence_Data_Bi3/'
out_path = 'WI3_BusinessIntelligence_Result_Bi3/'

In [2]:
# Collect files
file_collector = FileCollector()
day_to_sensor_files = file_collector.get_files_matched_to_date(input_path)
cycle_recognizer = CycleRecognizer()

In [ ]:
# Fill files with cycles
position_to_sensor_files = {'inside': [], 'outside': []}
for day_to_sensor_file in day_to_sensor_files:
    sensor_files = day_to_sensor_files[day_to_sensor_file]
    inside_file = file_collector.get_inside_file(sensor_files)
    outside_file = file_collector.get_outside_file(sensor_files)
    outside_file = cycle_recognizer.recognize_cycle(input_path+outside_file)
    cycle_columns = outside_file[['timer_minute', 'timer_second', 'sensor_cycle', 'sensor_cycle_similarity', 'cycle_number']].copy()
    inside_file = cycle_recognizer.append_cycle_to_file(cycle_columns, input_path+inside_file)
    inside_file = inside_file[['timer', 'timer_minute', 'timer_second', 'accelerometer_x_max', 'accelerometer_y_max', 'accelerometer_z_max', 'accelerometer_x_min', 'accelerometer_y_min', 'accelerometer_z_min',\
                              'accelerometer_x_max_quantile', 'accelerometer_y_max_quantile', 'accelerometer_z_max_quantile', 'accelerometer_x_min_quantile', 'accelerometer_y_min_quantile',\
                               'accelerometer_z_min_quantile', 'magnetometer_x_mean', 'magnetometer_y_mean', 'magnetometer_z_mean', 'magnetometer_x_median', 'magnetometer_y_median',\
                              'magnetometer_z_median', 'audio_max', 'audio_max_quantile', 'sensor_cycle_similarity', 'sensor_cycle', 'cycle_number']]
    outside_file = outside_file[['timer', 'timer_minute', 'timer_second', 'accelerometer_x_max', 'accelerometer_y_max', 'accelerometer_z_max', 'accelerometer_x_min', 'accelerometer_y_min', 'accelerometer_z_min',\
                              'accelerometer_x_max_quantile', 'accelerometer_y_max_quantile', 'accelerometer_z_max_quantile', 'accelerometer_x_min_quantile', 'accelerometer_y_min_quantile',\
                               'accelerometer_z_min_quantile', 'magnetometer_x_mean', 'magnetometer_y_mean', 'magnetometer_z_mean', 'magnetometer_x_median', 'magnetometer_y_median',\
                              'magnetometer_z_median', 'audio_max', 'audio_max_quantile', 'sensor_cycle_similarity', 'sensor_cycle', 'cycle_number']]
    position_to_sensor_files['inside'].append(inside_file)
    position_to_sensor_files['outside'].append(outside_file)

D:\Program Files\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# Iterate trough relevant files, add to main data frame, add shift and save as .csv
for sensor_files in position_to_sensor_files:
    final_dataframe = None
    for sensor_file in position_to_sensor_files[sensor_files]:
        shift_recognizer = ShiftRecognizer()
        sensor_dataframe = shift_recognizer.add_shift(sensor_file)
        if final_dataframe is None:
            final_dataframe = sensor_dataframe
        else:
            final_dataframe = pandas.concat([final_dataframe, sensor_dataframe])
    final_dataframe.to_csv(out_path+sensor_files+'.csv', index=False)